In [2]:
!pip install crewai crewai_tools langchain_community "crewai[tools]"

In [ ]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
import os
os.environ["OPENAI_API_KEY"] = "NA"

In [ ]:
llm = ChatOpenAI(
    model = "crewai-llama3",
    base_url = "http://localhost:11434"
    )

### Agent Attributes

Role: Defines the agent’s function within the crew. It determines the kind of tasks the agent is best suited for.

Goal: The individual objective that the agent aims to achieve. It guides the agent’s decision-making process.

Backstory: Provides context to the agent’s role and goal, enriching the interaction and collaboration dynamics.

LLM :(optional)Represents the language model that will run the agent. It dynamically fetches the model name from the OPENAI_MODEL_NAME environment variable, defaulting to "gpt-4" if not specified.

Tools :(optional)Set of capabilities or functions that the agent can use to perform tasks. Expected to be instances of custom classes compatible with the agent's execution environment. Tools are initialized with a default value of an empty list.

Function Calling LLM :(optional)Specifies the language model that will handle the tool calling for this agent, overriding the crew function calling LLM if passed. Default is None.

Max Iter :(optional)The maximum number of iterations the agent can perform before being forced to give its best answer. Default is 25.

Max RPM :(optional)The maximum number of requests per minute the agent can perform to avoid rate limits. It's optional and can be left unspecified, with a default value of None.

max_execution_time :(optional)Maximum execution time for an agent to execute a task It's optional and can be left unspecified, with a default value of None, menaning no max execution time

Verbose: (optional)Setting this to True configures the internal logger to provide detailed execution logs, aiding in debugging and monitoring. Default is False.

Allow Delegation: (optional)Agents can delegate tasks or questions to one another, ensuring that each task is handled by the most suitable agent. Default is True.

Step Callback: (optional)A function that is called after each step of the agent. This can be used to log the agent's actions or to perform other operations. It will overwrite the crew step_callback.

Cache: (optional)Indicates if the agent should use a cache for tool usage. Default is True

In [ ]:
### Content Planner Agent

planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article about the topic: {topic} in 'https://medium.com/'."
              "You collect information that helps the audience learn something and make informed decisions.\n"
              "You have to prepare a detailed outline and the relevant topics and sub-topics that has to be a part of the blog-post.\n"
              "Your work is the basis for the Content Writer to write an article on this topic.",
    llm=llm,
    allow_delegation=False,
 verbose=True
)

In [ ]:
### Content Writer Agent

writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate opinion piece about the topic: {topic}",
    backstory="You're working on a writing a new opinion piece about the topic: {topic} in 'https://medium.com/'.\n"
              "You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic.\n"
              "You follow the main objectives and direction of the outline, as provide by the Content Planner.\n"
              "You also provide objective and impartial insights and back them up with information provide by the Content Planner.\n"
              "You acknowledge in your opinion piece when your statements are opinions as opposed to objective statements.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

In [ ]:
### Content Editor Agent

editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with the writing style of the organization 'https://medium.com/'.",
    backstory="You are an editor who receives a blog post from the Content Writer.\n"
              "Your goal is to review the blog post to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints when providing opinions or assertions, and also avoids major controversial topics "
              "or opinions when possible.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

### Task Attributes

Description: A clear, concise statement of what the task entails.

Agent: The agent responsible for the task, assigned either directly or by the crew’s process.

Expected Output: A detailed description of what the task’s completion looks like.

Tools: (optional)The functions or capabilities the agent can utilize to perform the task.

Async Execution: (optional)If set, the task executes asynchronously, allowing progression without waiting for completion.

Context:(optional)Specifies tasks whose outputs are used as context for this task.

Config: (optional)Additional configuration details for the agent executing the task, allowing further customization.

Output JSON :(optional)Outputs a JSON object, requiring an OpenAI client. Only one output format can be set.

Output Pydantic :(optional)Outputs a Pydantic model object, requiring an OpenAI client. Only one output format can be set.

Output File: (optional)Saves the task output to a file. If used with Output JSON or Output Pydantic, specifies how the output is saved.

Callback:(optional)A Python callable that is executed with the task's output upon completion.

Human Input :(optional)Indicates if the task requires human feedback at the end, useful for tasks needing human oversight.

In [ ]:
### Create planner task

plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering their interests and pain points.\n"
        "3. Develop a detailed content outline including an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources.",
    agent=planner,
)

In [ ]:
### Create Writer Task

write = Task(
    description=(
        "1. Use the content plan to craft a compelling blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named in an engaging manner.\n"
        "4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [ ]:
### Create Editor Task
edit = Task(
    description=("Proofread the given blog post for grammatical errors and alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [ ]:
varname = """line 1 of text
             line 2 of text
          """

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [ ]:
inputs = {"topic":"Comparative study of LangGraph, Autogen and Crewai for building multi-agent system."}
result = crew.kickoff(inputs=inputs)